<a href="https://colab.research.google.com/github/Taedriel/ZSL-v2/blob/wordEmbedding/WordsEmbeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install transformers wget 
!pwd

/content


In [2]:
import wget, os

import torch
from transformers import BertTokenizer, BertModel

import logging
logging.basicConfig(level = logging.INFO)

import matplotlib.pyplot as plt
% matplotlib inline

In [ ]:
logging.info("Downloading dataset...")

url = ""

INFO:root:Downloading dataset...


In [34]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', padding=True, truncation=True,)

model = BertModel.from_pretrained("bert-base-uncased", output_hidden_states = True, )
model.eval()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [43]:
tags = ["After stealing money from the bank vault, the bank robber was seen " \
       "fishing on the Mississippi river bank"]

vectors = []

with open("vectors.csv", "w") as f:
    for tag in tags:

        inputs = tokenizer(tag, return_tensors = "pt")

        with torch.no_grad():
            outputs = model(**inputs)

        hidden_states = outputs[2]

        # print ("Number of layers:", len(hidden_states), "  (initial embeddings + 12 BERT layers)")
        # print ("Number of batches:", len(hidden_states[0]))
        print ("Number of tokens:", len(hidden_states[0][0]) - 2)
        # print ("Number of hidden units:", len(hidden_states[0][0][0]))


        # We got:
        # [# layers, # batches, # tokens, # features]
        # Desired dimensions:
        # [# tokens, # layers, # features]
        token_embeddings = torch.stack(hidden_states, dim=0)
        token_embeddings = torch.squeeze(token_embeddings, dim=1)
        token_embeddings = token_embeddings.permute(1,0,2)



        # apply different strategy to summarize word embeddings
        token_sum_4 = []
        for token in token_embeddings:

            sum_vec = torch.sum(token[-4:], dim = 0)
            token_sum_4.append(sum_vec)

        nbToken = len(hidden_states[0][0]) - 2
        tokenized_text = tokenizer.tokenize(tag)

        for i in range(len(tokenized_text)):

            realTag = tokenized_text[i]

            line = ";".join(list(map(str, map(float, token_sum_4[i+1][:]))))
            vectors.append((realTag, token_sum_4[i+1][:]))
            print(realTag, ";", line, sep="", file=f)


# compute cosine similarity between vectors
for j, vector in enumerate(vectors):

    for i, otherVector in enumerate(vectors):

        if i == j:
            continue
        
        cos = torch.nn.CosineSimilarity(dim=0)
        similarity = cos(vector[1], otherVector[1])

        print(vector[0], "^", otherVector[0], "=", similarity)


Number of tokens: 19
after ^ stealing = tensor(0.5974)
after ^ money = tensor(0.4088)
after ^ from = tensor(0.4426)
after ^ the = tensor(0.5708)
after ^ bank = tensor(0.3038)
after ^ vault = tensor(0.3225)
after ^ , = tensor(0.1957)
after ^ the = tensor(0.6123)
after ^ bank = tensor(0.3360)
after ^ robber = tensor(0.4193)
after ^ was = tensor(0.6301)
after ^ seen = tensor(0.5804)
after ^ fishing = tensor(0.3722)
after ^ on = tensor(0.3946)
after ^ the = tensor(0.4453)
after ^ mississippi = tensor(0.2043)
after ^ river = tensor(0.2568)
after ^ bank = tensor(0.2640)
stealing ^ after = tensor(0.5974)
stealing ^ money = tensor(0.5681)
stealing ^ from = tensor(0.5358)
stealing ^ the = tensor(0.5882)
stealing ^ bank = tensor(0.4186)
stealing ^ vault = tensor(0.5302)
stealing ^ , = tensor(0.1417)
stealing ^ the = tensor(0.5886)
stealing ^ bank = tensor(0.4402)
stealing ^ robber = tensor(0.6203)
stealing ^ was = tensor(0.5161)
stealing ^ seen = tensor(0.4477)
stealing ^ fishing = tensor(0.4226